In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parents[0]
sys.path.append(str(project_root))
import pandas as pd
from src import get_geoparse
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
expression_df, phenotype_df, gene_annot = get_geoparse()
phenotype_df["hours"] = (
    phenotype_df["title"]
    .str.extract(r"(\d+(?:\.\d+)?)\s*hours", expand=False)
    .astype(float)
)
phenotype_df["treated"] = ~phenotype_df["title"].str.contains(
    "negative control", case=False, na=False
)
phenotype_df = phenotype_df[['hours', 'treated']]

Precheck data for expression distribution

In [ ]:
plt.figure(figsize=(8, 7))
expression_df.hist(bins=100)
plt.title("Histogram of Samples")
plt.tight_layout()
plt.figure(figsize=(8, 7))
expression_df.boxplot()
plt.title("Boxplot of Samples")
plt.tight_layout()

PCA Analysis of Variance:
    Compare time and treatment to PC 1 & 2

In [ ]:
X = expression_df.T
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)
pca_df = pd.DataFrame(
    X_pca,
    index=X.index,
    columns=[f"PC{i+1}" for i in range(X_pca.shape[1])]
).join(phenotype_df)
pca_df['hours_cat'] = pca_df['hours'].astype(str)

plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=pca_df,
    x="PC1",
    y="PC2",
    hue="hours_cat",
    palette="viridis",
    s=80
)
plt.title("PCA of samples (colored by time)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=pca_df,
    x="PC1",
    y="PC2",
    hue="treated",
    style="treated",
    s=80
)
plt.title("PCA of samples (colored by treatment)")
plt.tight_layout()
plt.show()

Minimal Filtering:
    Only remove genes with pure background signal or low variance between samples

In [ ]:
MIN_EXPR = 4.5  # background cutoff
background_filter = (expression_df > MIN_EXPR).any(axis=1)
exp_filter = expression_df.loc[background_filter]
print(f"Genes after background filter: {exp_filter.shape[0]}")

MIN_VAR = 0.01
var_filter = exp_filter.var(axis=1) > MIN_VAR
exp_filter = exp_filter.loc[var_filter]
print(f"Genes after variance filter: {exp_filter.shape[0]}")
plt.hist(expression_df.values.flatten(), bins=100, alpha=0.5, label="All genes")
plt.hist(exp_filter.values.flatten(), bins=100, alpha=0.5, label="Filtered")
plt.legend()
plt.title("Expression distributions")
plt.show()